In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings                  #워닝 메시지
warnings.filterwarnings('ignore')
# 그래프에서 한글 깨짐 방지
import platform

if platform.system() == "Darwin":          # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)



---
- train-test split (2.3 / 7.7)

In [2]:
df=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/분리전_상장.csv',index_col=False, encoding='utf-8')

In [3]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(df.drop('t-1감사의견코드',axis=1),df['t-1감사의견코드'],test_size=0.23,stratify=df['t-1감사의견코드'],random_state=0) 

In [4]:
train_data = pd.concat([X_train,y_train], axis=1)

In [5]:
test_data= pd.concat([X_test,y_test],axis=1 )

#### 

In [6]:
# # 데이터프레임에서 '이자보상배율' 열의 1퍼센트 절사평균과 표준편차 계산
# def trimmed_mean_1(x):
#     trim_pct = 0.1
#     lower_bound = np.percentile(x, trim_pct * 100)
#     upper_bound = np.percentile(x, (1 - trim_pct) * 100)
#     trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
#     return trimmed_values.mean()

# def trimmed_std_1(x):
#     trim_pct = 0.1
#     lower_bound = np.percentile(x, trim_pct * 100)
#     upper_bound = np.percentile(x, (1 - trim_pct) * 100)
#     trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
#     return trimmed_values.std()

# train_data['평균이보배'] = train_data.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)
# train_data['이보배S.E'] = train_data.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_std_1)

# # 개별 이자보상배율과 평균-표준편차를 비교하여 이보배초과여부 계산
# train_data['이보배초과여부'] = np.where(train_data['이자보상배율'] < (train_data['평균이보배'] - 10*train_data['이보배S.E']), 1, 0)


In [7]:
# 1퍼센트 절사평균 계산 함수
def trimmed_mean_1(x):
    trim_pct = 0.05
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

# 1퍼센트 절사표준편차 계산 함수
def trimmed_std_1(x):
    trim_pct = 0.05
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.std()


---
# 1.파생이자보상배율

In [8]:

# train_data에서 산업분류와 연도별로 평균과 표준편차를 계산하여 딕셔너리로 저장
mean_std_dict = train_data.groupby(['산업분류', 'Year'])['이자보상배율'].agg([('평균이보배', trimmed_mean_1), ('이보배S.E', trimmed_std_1)]).reset_index()

# 표준편차를 그룹바이된 개수로 나누는 함수 추가
def divide_by_group_size(x):
    return x / x.count()

# train_data에서 그룹바이된 개수로 이보배 S.E를 나누어 '이보배S.E' 컬럼 생성
train_data['이보배S.E'] = train_data.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_std_1) / train_data.groupby(['산업분류', 'Year'])['이자보상배율'].transform(divide_by_group_size)

# test_data에서도 그룹바이된 개수로 이보배 S.E를 나누도록 업데이트
test_data['이보배S.E'] = test_data.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_std_1) / test_data.groupby(['산업분류', 'Year'])['이자보상배율'].transform(divide_by_group_size)

# train_data에서 '평균이보배'와 '이보배초과여부' 컬럼 생성
def apply_mean_std(row):
    industry = row['산업분류']
    year = row['Year']
    mean_std_filtered = mean_std_dict[(mean_std_dict['산업분류'] == industry) & (mean_std_dict['Year'] == year)]
    if len(mean_std_filtered) > 0:
        mean_val = mean_std_filtered['평균이보배'].iloc[0]
        std_val = mean_std_filtered['이보배S.E'].iloc[0]
    else:
        mean_val = np.nan
        std_val = np.nan
    return pd.Series([mean_val, std_val])

train_data[['평균이보배', '이보배S.E']] = train_data.apply(apply_mean_std, axis=1)
test_data[['평균이보배', '이보배S.E']] = test_data.apply(apply_mean_std, axis=1)

# 개별 이자보상배율과 평균-표준편차를 비교하여 '이보배초과여부' 컬럼 생성
train_data['이보배초과여부'] = np.where(train_data['이자보상배율'] < (train_data['평균이보배'] - train_data['이보배S.E']), 1, 0)
test_data['이보배초과여부'] = np.where(test_data['이자보상배율'] < (test_data['평균이보배'] - test_data['이보배S.E']), 1, 0)

In [9]:
train_data[['회사명','거래소코드','산업분류','Year','이자보상배율', '평균이보배', '이보배S.E','이보배초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,이자보상배율,평균이보배,이보배S.E,이보배초과여부
11836,알톤스포츠,123750.0,제조업,2014.0,20.378314,39.505353,136.548179,0
2119,대림통상,6570.0,제조업,2013.0,0.227011,23.304924,58.075852,0
5453,제이와이피엔터테인먼트,35900.0,정보통신업,2017.0,6057.061304,996.132290,2172.800846,0
1225,쌍용씨앤이,3410.0,제조업,2015.0,5.131511,47.491407,168.868119,0
12589,필옵틱스,161580.0,제조업,2021.0,-4.460295,21.251855,50.945084,0
2345,이수페타시스,7660.0,제조업,2012.0,4.614500,14.628016,32.344352,0
209,가온전선,500.0,제조업,2013.0,3.239944,23.304924,58.075852,0
914,제일파마홀딩스,2620.0,"전문, 과학 및 기술 서비스업",2021.0,-2.726263,16.069131,34.980689,0
9786,동양고속,84670.0,운수 및 창고업,2015.0,5.396484,5.106638,6.130102,0
7404,프리엠스,53160.0,제조업,2013.0,182.441441,23.304924,58.075852,0


In [10]:
test_data[['회사명','거래소코드','산업분류','Year','이자보상배율', '평균이보배','이보배S.E', '이보배초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,이자보상배율,평균이보배,이보배S.E,이보배초과여부
5266,에스비에스,34120.0,정보통신업,2020.0,10.469744,47.750604,102.486769,0
10422,현우산업,92300.0,제조업,2020.0,0.048809,15.175868,35.936237,0
10518,후성,93370.0,제조업,2011.0,4.792808,17.275761,41.162050,0
10628,이엠코리아,95190.0,제조업,2014.0,1.613751,39.505353,136.548179,0
8533,멀티캠퍼스,67280.0,교육 서비스업,2019.0,28.669141,20.409508,11.294307,0
6613,코스맥스비티아이,44820.0,"전문, 과학 및 기술 서비스업",2019.0,-0.477778,12.631260,33.876745,0
7356,아이앤씨테크놀로지,52860.0,제조업,2014.0,-45.054287,39.505353,136.548179,0
1708,한창,5110.0,도매 및 소매업,2011.0,-2.764420,435.625294,1529.133987,0
6509,국순당,43650.0,제조업,2020.0,261.784364,15.175868,35.936237,0
3858,우신시스템,17370.0,제조업,2013.0,3.087718,23.304924,58.075852,0


In [11]:
train_data['이보배초과여부'].value_counts()


이보배초과여부
0    11013
1      317
Name: count, dtype: int64

In [12]:
train_data[train_data['이보배초과여부']==1]['t-1감사의견코드'].value_counts()

t-1감사의견코드
0.0    312
1.0      5
Name: count, dtype: int64

---
# 2. 파부비

In [13]:

# train_data에서 산업분류와 연도별로 평균과 표준편차를 계산하여 딕셔너리로 저장
mean_std_dict = train_data.groupby(['산업분류', 'Year'])['부채비율'].agg([('평균파부비', trimmed_mean_1), ('파부비S.E', trimmed_std_1)]).reset_index()

# 표준편차를 그룹바이된 개수로 나누는 함수 추가
def divide_by_group_size(x):
    return x / x.count()

# train_data에서 그룹바이된 개수로 이보배 S.E를 나누어 '파부비S.E' 컬럼 생성
train_data['파부비S.E'] = train_data.groupby(['산업분류', 'Year'])['부채비율'].transform(trimmed_std_1) / train_data.groupby(['산업분류', 'Year'])['부채비율'].transform(divide_by_group_size)

# test_data에서도 그룹바이된 개수로 이보배 S.E를 나누도록 업데이트
test_data['파부비S.E'] = test_data.groupby(['산업분류', 'Year'])['부채비율'].transform(trimmed_std_1) / test_data.groupby(['산업분류', 'Year'])['부채비율'].transform(divide_by_group_size)

# train_data에서 '평균파부비'와 '이보배초과여부' 컬럼 생성
def apply_mean_std(row):
    industry = row['산업분류']
    year = row['Year']
    mean_std_filtered = mean_std_dict[(mean_std_dict['산업분류'] == industry) & (mean_std_dict['Year'] == year)]
    if len(mean_std_filtered) > 0:
        mean_val = mean_std_filtered['평균파부비'].iloc[0]
        std_val = mean_std_filtered['파부비S.E'].iloc[0]
    else:
        mean_val = np.nan
        std_val = np.nan
    return pd.Series([mean_val, std_val])

train_data[['평균파부비', '파부비S.E']] = train_data.apply(apply_mean_std, axis=1)
test_data[['평균파부비', '파부비S.E']] = test_data.apply(apply_mean_std, axis=1)

# 개별 부채비율과 평균-표준편차를 비교하여 '이보배초과여부' 컬럼 생성
train_data['파부비초과여부'] = np.where(train_data['부채비율'] > (train_data['평균파부비'] + train_data['파부비S.E']), 1, 0)
test_data['파부비초과여부'] = np.where(test_data['부채비율'] > (test_data['평균파부비'] + test_data['파부비S.E']), 1, 0)



In [14]:
train_data[['회사명','거래소코드','산업분류','Year','부채비율', '평균파부비', '파부비S.E','파부비초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,부채비율,평균파부비,파부비S.E,파부비초과여부
11836,알톤스포츠,123750.0,제조업,2014.0,37.76,111.366743,78.239023,0
2119,대림통상,6570.0,제조업,2013.0,93.11,108.526194,73.547979,0
5453,제이와이피엔터테인먼트,35900.0,정보통신업,2017.0,45.09,58.019658,46.010334,0
1225,쌍용씨앤이,3410.0,제조업,2015.0,117.65,100.191208,75.111738,0
12589,필옵틱스,161580.0,제조업,2021.0,181.32,92.696314,65.634751,1
2345,이수페타시스,7660.0,제조업,2012.0,102.11,108.079100,73.055590,0
209,가온전선,500.0,제조업,2013.0,99.99,108.526194,73.547979,0
914,제일파마홀딩스,2620.0,"전문, 과학 및 기술 서비스업",2021.0,78.37,89.878133,59.004240,0
9786,동양고속,84670.0,운수 및 창고업,2015.0,82.91,249.602353,284.318463,0
7404,프리엠스,53160.0,제조업,2013.0,12.29,108.526194,73.547979,0


In [15]:
test_data[['회사명','거래소코드','산업분류','Year','부채비율', '평균파부비', '파부비S.E','파부비초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,부채비율,평균파부비,파부비S.E,파부비초과여부
5266,에스비에스,34120.0,정보통신업,2020.0,106.43,57.887260,47.603003,1
10422,현우산업,92300.0,제조업,2020.0,131.38,93.922031,68.646025,0
10518,후성,93370.0,제조업,2011.0,140.70,112.267716,68.308006,0
10628,이엠코리아,95190.0,제조업,2014.0,166.51,111.366743,78.239023,0
8533,멀티캠퍼스,67280.0,교육 서비스업,2019.0,81.81,97.070000,33.771420,0
6613,코스맥스비티아이,44820.0,"전문, 과학 및 기술 서비스업",2019.0,150.77,102.318082,64.110277,0
7356,아이앤씨테크놀로지,52860.0,제조업,2014.0,28.39,111.366743,78.239023,0
1708,한창,5110.0,도매 및 소매업,2011.0,61.15,104.359577,69.680943,0
6509,국순당,43650.0,제조업,2020.0,10.06,93.922031,68.646025,0
3858,우신시스템,17370.0,제조업,2013.0,159.88,108.526194,73.547979,0


In [16]:
train_data['파부비초과여부'].value_counts()

파부비초과여부
0    9064
1    2266
Name: count, dtype: int64

In [17]:
train_data[train_data['파부비초과여부']==1]['t-1감사의견코드'].value_counts()

t-1감사의견코드
0.0    2243
1.0      23
Name: count, dtype: int64

---
# 3.파당비


In [18]:

# train_data에서 산업분류와 연도별로 평균과 표준편차를 계산하여 딕셔너리로 저장
mean_std_dict = train_data.groupby(['산업분류', 'Year'])['당좌비율'].agg([('평균파당비', trimmed_mean_1), ('파당비S.E', trimmed_std_1)]).reset_index()

# 표준편차를 그룹바이된 개수로 나누는 함수 추가
def divide_by_group_size(x):
    return x / x.count()

# train_data에서 그룹바이된 개수로 이보배 S.E를 나누어 '파당비S.E' 컬럼 생성
train_data['파당비S.E'] = train_data.groupby(['산업분류', 'Year'])['당좌비율'].transform(trimmed_std_1) / train_data.groupby(['산업분류', 'Year'])['당좌비율'].transform(divide_by_group_size)

# test_data에서도 그룹바이된 개수로 이보배 S.E를 나누도록 업데이트
test_data['파당비S.E'] = test_data.groupby(['산업분류', 'Year'])['당좌비율'].transform(trimmed_std_1) / test_data.groupby(['산업분류', 'Year'])['당좌비율'].transform(divide_by_group_size)

# train_data에서 '평균파당비'와 '이보배초과여부' 컬럼 생성
def apply_mean_std(row):
    industry = row['산업분류']
    year = row['Year']
    mean_std_filtered = mean_std_dict[(mean_std_dict['산업분류'] == industry) & (mean_std_dict['Year'] == year)]
    if len(mean_std_filtered) > 0:
        mean_val = mean_std_filtered['평균파당비'].iloc[0]
        std_val = mean_std_filtered['파당비S.E'].iloc[0]
    else:
        mean_val = np.nan
        std_val = np.nan
    return pd.Series([mean_val, std_val])

train_data[['평균파당비', '파당비S.E']] = train_data.apply(apply_mean_std, axis=1)
test_data[['평균파당비', '파당비S.E']] = test_data.apply(apply_mean_std, axis=1)

# 개별 당좌비율과 평균-표준편차를 비교하여 '이보배초과여부' 컬럼 생성
train_data['파당비초과여부'] = np.where(train_data['당좌비율'] < (train_data['평균파당비'] - train_data['파당비S.E']), 1, 0)
test_data['파당비초과여부'] = np.where(test_data['당좌비율'] < (test_data['평균파당비'] - test_data['파당비S.E']), 1, 0)



In [19]:
train_data[['회사명','거래소코드','산업분류','Year','당좌비율', '평균파당비', '파당비S.E','파당비초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,당좌비율,평균파당비,파당비S.E,파당비초과여부
11836,알톤스포츠,123750.0,제조업,2014.0,167.87,141.935505,107.555804,0
2119,대림통상,6570.0,제조업,2013.0,89.05,156.104494,120.848475,0
5453,제이와이피엔터테인먼트,35900.0,정보통신업,2017.0,147.50,311.897265,227.416285,0
1225,쌍용씨앤이,3410.0,제조업,2015.0,24.76,175.676953,147.121089,1
12589,필옵틱스,161580.0,제조업,2021.0,33.98,176.204042,141.722213,1
2345,이수페타시스,7660.0,제조업,2012.0,189.85,141.340294,100.123503,0
209,가온전선,500.0,제조업,2013.0,173.03,156.104494,120.848475,0
914,제일파마홀딩스,2620.0,"전문, 과학 및 기술 서비스업",2021.0,37.52,366.998400,448.850802,0
9786,동양고속,84670.0,운수 및 창고업,2015.0,44.98,78.045294,45.824187,0
7404,프리엠스,53160.0,제조업,2013.0,605.98,156.104494,120.848475,0


In [20]:
test_data[['회사명','거래소코드','산업분류','Year','당좌비율', '평균파당비', '파당비S.E','파당비초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,당좌비율,평균파당비,파당비S.E,파당비초과여부
5266,에스비에스,34120.0,정보통신업,2020.0,235.67,303.102534,239.458452,0
10422,현우산업,92300.0,제조업,2020.0,100.56,189.029958,170.913244,0
10518,후성,93370.0,제조업,2011.0,46.88,122.752091,77.060070,0
10628,이엠코리아,95190.0,제조업,2014.0,33.17,141.935505,107.555804,1
8533,멀티캠퍼스,67280.0,교육 서비스업,2019.0,134.14,104.095000,11.745044,0
6613,코스맥스비티아이,44820.0,"전문, 과학 및 기술 서비스업",2019.0,57.86,211.017260,246.302848,0
7356,아이앤씨테크놀로지,52860.0,제조업,2014.0,561.94,141.935505,107.555804,0
1708,한창,5110.0,도매 및 소매업,2011.0,242.18,144.384366,111.073730,0
6509,국순당,43650.0,제조업,2020.0,562.81,189.029958,170.913244,0
3858,우신시스템,17370.0,제조업,2013.0,118.31,156.104494,120.848475,0


In [21]:
train_data['파당비초과여부'].value_counts()

파당비초과여부
0    10851
1      479
Name: count, dtype: int64

In [22]:
train_data[train_data['파당비초과여부']==1]['t-1감사의견코드'].value_counts()

t-1감사의견코드
0.0    473
1.0      6
Name: count, dtype: int64

---
# 4.파차의

In [23]:
# train_data에서 산업분류와 연도별로 평균과 표준편차를 계산하여 딕셔너리로 저장
mean_std_dict = train_data.groupby(['산업분류', 'Year'])['차입금의존도'].agg([('평균파차의', trimmed_mean_1), ('파차의S.E', trimmed_std_1)]).reset_index()

# 표준편차를 그룹바이된 개수로 나누는 함수 추가
def divide_by_group_size(x):
    return x / x.count()

# train_data에서 그룹바이된 개수로 이보배 S.E를 나누어 '파차의S.E' 컬럼 생성
train_data['파차의S.E'] = train_data.groupby(['산업분류', 'Year'])['차입금의존도'].transform(trimmed_std_1) / train_data.groupby(['산업분류', 'Year'])['차입금의존도'].transform(divide_by_group_size)

# test_data에서도 그룹바이된 개수로 이보배 S.E를 나누도록 업데이트
test_data['파차의S.E'] = test_data.groupby(['산업분류', 'Year'])['차입금의존도'].transform(trimmed_std_1) / test_data.groupby(['산업분류', 'Year'])['차입금의존도'].transform(divide_by_group_size)

# train_data에서 '평균파차의'와 '이보배초과여부' 컬럼 생성
def apply_mean_std(row):
    industry = row['산업분류']
    year = row['Year']
    mean_std_filtered = mean_std_dict[(mean_std_dict['산업분류'] == industry) & (mean_std_dict['Year'] == year)]
    if len(mean_std_filtered) > 0:
        mean_val = mean_std_filtered['평균파차의'].iloc[0]
        std_val = mean_std_filtered['파차의S.E'].iloc[0]
    else:
        mean_val = np.nan
        std_val = np.nan
    return pd.Series([mean_val, std_val])

train_data[['평균파차의', '파차의S.E']] = train_data.apply(apply_mean_std, axis=1)
test_data[['평균파차의', '파차의S.E']] = test_data.apply(apply_mean_std, axis=1)

# 개별 차입금의존도과 평균-표준편차를 비교하여 '이보배초과여부' 컬럼 생성
train_data['파차의초과여부'] = np.where(train_data['차입금의존도'] > (train_data['평균파차의'] + train_data['파차의S.E']), 1, 0)
test_data['파차의초과여부'] = np.where(test_data['차입금의존도'] > (test_data['평균파차의'] + test_data['파차의S.E']), 1, 0)



In [24]:
train_data[['회사명','거래소코드','산업분류','Year','차입금의존도', '평균파차의', '파차의S.E','파차의초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,차입금의존도,평균파차의,파차의S.E,파차의초과여부
11836,알톤스포츠,123750.0,제조업,2014.0,15.27,23.804847,16.243508,0
2119,대림통상,6570.0,제조업,2013.0,34.26,25.901818,15.758640,0
5453,제이와이피엔터테인먼트,35900.0,정보통신업,2017.0,0.00,9.051774,10.766085,0
1225,쌍용씨앤이,3410.0,제조업,2015.0,32.28,22.302488,15.856632,0
12589,필옵틱스,161580.0,제조업,2021.0,34.56,22.513197,13.351661,0
2345,이수페타시스,7660.0,제조업,2012.0,30.01,25.378121,15.307717,0
209,가온전선,500.0,제조업,2013.0,26.87,25.901818,15.758640,0
914,제일파마홀딩스,2620.0,"전문, 과학 및 기술 서비스업",2021.0,12.89,18.776667,13.495276,0
9786,동양고속,84670.0,운수 및 창고업,2015.0,27.63,33.742941,19.567838,0
7404,프리엠스,53160.0,제조업,2013.0,0.95,25.901818,15.758640,0


In [25]:
test_data[['회사명','거래소코드','산업분류','Year','차입금의존도', '평균파차의', '파차의S.E','파차의초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,차입금의존도,평균파차의,파차의S.E,파차의초과여부
5266,에스비에스,34120.0,정보통신업,2020.0,25.45,13.061370,11.395943,1
10422,현우산업,92300.0,제조업,2020.0,34.68,22.921043,14.055252,0
10518,후성,93370.0,제조업,2011.0,45.04,25.672996,14.517386,1
10628,이엠코리아,95190.0,제조업,2014.0,51.12,23.804847,16.243508,1
8533,멀티캠퍼스,67280.0,교육 서비스업,2019.0,17.56,27.340000,6.279108,0
6613,코스맥스비티아이,44820.0,"전문, 과학 및 기술 서비스업",2019.0,46.24,22.948082,14.805376,1
7356,아이앤씨테크놀로지,52860.0,제조업,2014.0,13.18,23.804847,16.243508,0
1708,한창,5110.0,도매 및 소매업,2011.0,9.13,23.124800,16.954785,0
6509,국순당,43650.0,제조업,2020.0,0.70,22.921043,14.055252,0
3858,우신시스템,17370.0,제조업,2013.0,21.28,25.901818,15.758640,0


In [26]:
train_data['파차의초과여부'].value_counts()

파차의초과여부
0    8692
1    2638
Name: count, dtype: int64

In [27]:
train_data[train_data['파차의초과여부']==1]['t-1감사의견코드'].value_counts()

t-1감사의견코드
0.0    2611
1.0      27
Name: count, dtype: int64

In [28]:
# # 차입금의존도 30% 이상 기업을 1(부실), 아니면 0(정상)

# train_data['차입금의존정도'] = 0

# for i in train_data.index :
#     if train_data['차입금의존도'][i] > 30 :
#         train_data['차입금의존정도'][i] = 1
#     else :
#         train_data['차입금의존정도'][i] = 0

In [29]:
# train_data['차입금의존정도'].value_counts()

In [30]:
# train_data[train_data['차입금의존정도']==1]['t-1감사의견코드'].value_counts()

---
# 5.파로이

In [31]:

# train_data에서 산업분류와 연도별로 평균과 표준편차를 계산하여 딕셔너리로 저장
mean_std_dict = train_data.groupby(['산업분류', 'Year'])['자기자본순이익률'].agg([('평균파로이', trimmed_mean_1), ('파로이S.E', trimmed_std_1)]).reset_index()

# 표준편차를 그룹바이된 개수로 나누는 함수 추가
def divide_by_group_size(x):
    return x / x.count()

# train_data에서 그룹바이된 개수로 이보배 S.E를 나누어 '파로이S.E' 컬럼 생성
train_data['파로이S.E'] = train_data.groupby(['산업분류', 'Year'])['자기자본순이익률'].transform(trimmed_std_1) / train_data.groupby(['산업분류', 'Year'])['자기자본순이익률'].transform(divide_by_group_size)

# test_data에서도 그룹바이된 개수로 이보배 S.E를 나누도록 업데이트
test_data['파로이S.E'] = test_data.groupby(['산업분류', 'Year'])['자기자본순이익률'].transform(trimmed_std_1) / test_data.groupby(['산업분류', 'Year'])['자기자본순이익률'].transform(divide_by_group_size)

# train_data에서 '평균파로이'와 '이보배초과여부' 컬럼 생성
def apply_mean_std(row):
    industry = row['산업분류']
    year = row['Year']
    mean_std_filtered = mean_std_dict[(mean_std_dict['산업분류'] == industry) & (mean_std_dict['Year'] == year)]
    if len(mean_std_filtered) > 0:
        mean_val = mean_std_filtered['평균파로이'].iloc[0]
        std_val = mean_std_filtered['파로이S.E'].iloc[0]
    else:
        mean_val = np.nan
        std_val = np.nan
    return pd.Series([mean_val, std_val])

train_data[['평균파로이', '파로이S.E']] = train_data.apply(apply_mean_std, axis=1)
test_data[['평균파로이', '파로이S.E']] = test_data.apply(apply_mean_std, axis=1)

# 개별 자기자본순이익률과 평균-표준편차를 비교하여 '이보배초과여부' 컬럼 생성
train_data['파로이초과여부'] = np.where(train_data['자기자본순이익률'] < (train_data['평균파로이'] - train_data['파로이S.E']), 1, 0)
test_data['파로이초과여부'] = np.where(test_data['자기자본순이익률'] < (test_data['평균파로이'] - test_data['파로이S.E']), 1, 0)



In [32]:
train_data[['회사명','거래소코드','산업분류','Year','자기자본순이익률', '평균파로이', '파로이S.E','파로이초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,자기자본순이익률,평균파로이,파로이S.E,파로이초과여부
11836,알톤스포츠,123750.0,제조업,2014.0,12.44,4.447276,10.507975,0
2119,대림통상,6570.0,제조업,2013.0,-5.36,4.818300,11.053347,0
5453,제이와이피엔터테인먼트,35900.0,정보통신업,2017.0,21.38,2.100855,13.131558,0
1225,쌍용씨앤이,3410.0,제조업,2015.0,6.10,4.689527,10.711980,0
12589,필옵틱스,161580.0,제조업,2021.0,-4.62,2.758375,13.772688,0
2345,이수페타시스,7660.0,제조업,2012.0,12.45,4.823366,10.484396,0
209,가온전선,500.0,제조업,2013.0,3.27,4.818300,11.053347,0
914,제일파마홀딩스,2620.0,"전문, 과학 및 기술 서비스업",2021.0,-0.85,0.617200,15.883789,0
9786,동양고속,84670.0,운수 및 창고업,2015.0,12.43,0.877647,14.795287,0
7404,프리엠스,53160.0,제조업,2013.0,8.46,4.818300,11.053347,0


In [33]:
test_data[['회사명','거래소코드','산업분류','Year','자기자본순이익률', '평균파로이', '파로이S.E','파로이초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,자기자본순이익률,평균파로이,파로이S.E,파로이초과여부
5266,에스비에스,34120.0,정보통신업,2020.0,-12.32,2.424110,16.642893,0
10422,현우산업,92300.0,제조업,2020.0,1.49,-0.464562,13.998166,0
10518,후성,93370.0,제조업,2011.0,12.20,6.089375,10.261218,0
10628,이엠코리아,95190.0,제조업,2014.0,2.38,4.447276,10.507975,0
8533,멀티캠퍼스,67280.0,교육 서비스업,2019.0,18.19,12.130000,0.098995,0
6613,코스맥스비티아이,44820.0,"전문, 과학 및 기술 서비스업",2019.0,0.13,-3.886027,19.922738,0
7356,아이앤씨테크놀로지,52860.0,제조업,2014.0,-23.56,4.447276,10.507975,1
1708,한창,5110.0,도매 및 소매업,2011.0,-29.43,1.852113,12.981926,1
6509,국순당,43650.0,제조업,2020.0,8.89,-0.464562,13.998166,0
3858,우신시스템,17370.0,제조업,2013.0,5.25,4.818300,11.053347,0


In [34]:
train_data['파로이초과여부'].value_counts()

파로이초과여부
0    9364
1    1966
Name: count, dtype: int64

In [35]:
train_data[train_data['파로이초과여부']==1]['t-1감사의견코드'].value_counts()

t-1감사의견코드
0.0    1926
1.0      40
Name: count, dtype: int64

In [39]:
# train_data.to_csv('train_상장.csv',index=False,encoding='euc-kr')

In [41]:
# test_data.to_csv('test_상장.csv',index=False,encoding='euc-kr')